In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd '/contet/drive/My Drive/Colab Notebooks'

[Errno 2] No such file or directory: '/contet/drive/My Drive/Colab Notebooks'
/content


In [6]:
ls

drive/  sample_data/


In [7]:
cd 'drive'

/content/drive


In [8]:
ls

'My Drive'/


In [9]:
cd 'My Drive'

/content/drive/My Drive


In [10]:
ls

'Colab Notebooks'/   pliki/


In [11]:
cd 'Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [12]:
ls

ColabDriveGithub.ipynb  dw_matrix/


In [13]:
cd 'dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [14]:
ls

data/  HeloGithub.ipynb  LICENSE  matrix_one/  README.md


In [15]:
ls data

shoes_prices.csv


In [46]:
df = pd.read_csv('data/shoes_prices.csv', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18280 entries, 0 to 18279
Data columns (total 48 columns):
id                     18280 non-null object
asins                  2161 non-null object
brand                  18263 non-null object
categories             18280 non-null object
colors                 10344 non-null object
count                  0 non-null float64
dateadded              18280 non-null object
dateupdated            18280 non-null object
descriptions           9704 non-null object
dimension              2968 non-null object
ean                    9690 non-null float64
features               13299 non-null object
flavors                0 non-null float64
imageurls              17255 non-null object
isbn                   0 non-null float64
keys                   18280 non-null object
manufacturer           6296 non-null object
manufacturernumber     14309 non-null object
merchants              13018 non-null object
name                   18280 non-null object
pric

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [51]:
df['brand_cat'] = df['brand'].factorize()[0]
df['brand_cat'].head()
run_model(['brand_cat'])  

(-58.38655694633361, 4.223555478221712)

In [57]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'])  

(-58.133398968282776, 4.206122611474276)

In [56]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])  

(-58.133398968282776, 4.206122611474276)

In [0]:
df.features.head().values
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [64]:
literal_eval(str_dict)[0]['key']

'Gender'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict
df['features_parsed'] = df['features'].map(parse_features)

In [73]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [75]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [78]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [79]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_shoe category', 'feat_leather grade', 'feat_waist',
       'feat_band width', 'feat_eyewear frame style',
       'feat_country of manufacture', 'feat_fuel type',
       'feat_is dark sky-compliant', 'feat_inseam', 'feat_date'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0]  * 100



In [90]:
df.shape[0]

18280

In [91]:
{k:v for k,v in keys_stat.items() if v > 50}

{'gender': 50.17505470459519}

In [92]:
{k:v for k,v in keys_stat.items() if v > 40}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519}

In [93]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]


for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [113]:
df[df.brand == df.feat_brand]. shape

(8846, 531)

In [138]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)

feats = ['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat',  'feat_style_cat', 'feat_design_cat',
 'feat_charge time (minutes)_cat',
 'feat_material composition_cat',
 'feat_material type_cat',
 'feat_product type_cat',
 'feat_lens socket width_cat',
 'feat_special features_cat',
 'feat_has paper wood_cat',
 'feat_variant group id_cat',
 'feat_watch case shape_cat',
 'feat_assembled product weight_cat',
 'feat_neckline_cat',
 'feat_is water-resistant_cat',
 'feat_count_cat',
 'feat_includes_cat',
 'feat_item width_cat',
 'feat_expandable_cat',
 'feat_sku#_cat',
 'feat_diameter_cat',
 'feat_release_cat',
 'feat_occasion_cat',
 'feat_sku number_cat',
 'feat_footwear type_cat',
 'feat_retail price_cat',
 'feat_fits over eyeglasses_cat',
 'feat_front style_cat',
 'feat_leg_cat',
 'feat_productgroup_cat',
 'feat_carats_cat',
 'feat_primary color_cat',
 'feat_sleeve style_cat',
 'feat_fabric care_cat',
 'feat_removable liner_cat',
 'feat_motion control_cat',
 'feat_bed size_cat',
 'feat_rx-able_cat',
 'feat_toe type_cat']
#feats += feats_cat
#feats = list(set(feats))#

run_model(feats, model)

(-57.34369569507636, 4.406001087524976)

In [139]:
X = df[ feats ].values
y = df['prices_amountmin'].values 

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm=PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)


Weight,Feature
0.2573 ± 0.0109,brand_cat
0.1035 ± 0.0041,feat_material_cat
0.0425 ± 0.0021,feat_gender_cat
0.0200 ± 0.0015,feat_brand_cat
0.0050 ± 0.0007,feat_style_cat
0.0042 ± 0.0008,feat_occasion_cat
0.0037 ± 0.0003,feat_watch case shape_cat
0.0005 ± 0.0001,feat_product type_cat
0.0002 ± 0.0000,feat_diameter_cat
0.0001 ± 0.0000,feat_design_cat


In [123]:
df['brand'].value_counts(normalize=True)

nike             0.097210
puma             0.033315
ralph lauren     0.028775
vans             0.021116
new balance      0.020295
                   ...   
vans av78        0.000055
a. testoni       0.000055
louis raphael    0.000055
capsmith         0.000055
therafirm        0.000055
Name: brand, Length: 1732, dtype: float64

In [125]:
df[ df['brand'] == 'nike' ].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [133]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_location - city/state',
 'feat_fabrication',
 'feat_location - country',
 'feat_catalog',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_certifications and listings',
 'feat_shoe category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_bracelet style_cat',
 'feat_sleeve length_cat',
 'feat_design_cat',
 'feat_charge time (minutes)_cat',
 'feat_material composition_cat',
 'feat_material type_cat',
 'feat_product type_cat',
 'feat_lens socket width_cat',
 'feat_special features_cat',
 'feat_has paper wood_cat',
 'feat_variant group id_cat',
 'feat_watch case shape_cat',
 'feat_assembled product weight_cat',
 'feat_neckline_cat',
 'feat_is water-resistant_cat',
 'feat_count_cat',
 'feat_includes_cat',
 'feat_item width_cat',
 'feat_expandable_cat',
 'feat_sku#_cat',
 'feat_diameter_cat',
 'fe

In [144]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [147]:
ls matrix_one/

day3.ipynb  day4.ipynb  day5.ipynb


In [0]:
!git add matrix_one/day5.ipynb

In [149]:
!git commit -m "my first machine learning scripts"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@43fd94b5e54c.(none)')


In [0]:
!git config --global user.email "lat.lukasz@gmail.com"
!git config --global user.name "Lukasz"

In [151]:
!git push -u origin master

Branch 'master' set up to track remote branch 'master' from 'origin'.
Everything up-to-date
